In [ ]:
#01. add different enemies - fly
fly_surf = pygame.image.load('graphics/fly/fly1.png').convert_alpha()

if event.type == enemy_timer:
    if randint(0, 2):
        enemy_rect_list.append(snail_surface.get_rect(bottomright = (randint(900, 1100), 300)))
    else:
        enemy_rect_list.append(fly_surf.get_rect(bottomright = (randint(900, 1100), 180)))

def enemy_movement(enmy_rct_list):
    if enmy_rct_list:  # list is not null
        for enmy_rct in enmy_rct_list:
            enmy_rct.x -= 5
            if enmy_rct.bottom == 300: win.blit(snail_surface, enmy_rct)
            else: win.blit(fly_surf, enmy_rct)
            
        enmy_rct_list = [enmy for enmy in enmy_rct_list if enmy.right > 0]

In [ ]:
#02. fix the collision logic

def isnt_Collide(plyr_rect, enmy_rect_list):
    if enmy_rect_list:
        for enmy_rect in enmy_rect_list:
            if plyr_rect.colliderect(enmy_rect): return False
    return True

# main loop
gameActive = isnt_Collide(player_rect, enemy_rect_list)

# game over
enemy_rect_list.clear()
player_rect.midbottom = (80, 300)
player_gravity = 0

In [ ]:
#03. more surfaces

player_walk_1 = pygame.image.load('graphics/player/player_walk_1.png').convert_alpha()
player_walk_2 = pygame.image.load('graphics/player/player_walk_2.png').convert_alpha()
player_walk = [player_walk_1, player_walk_2]
plyr_wlk_idx = 0

player_jump = pygame.image.load('graphics/player/jump.png').convert_alpha()

player_surf = player_walk[plyr_wlk_idx]

In [ ]:
#04. make a player animation function
# walk: player is on the ground
# jump: player is not on the ground
def player_animation():
    global player_surf, plyr_wlk_idx
    
    if player_rect.bottom < 300:
        player_surf = player_jump
    else:
        plyr_wlk_idx += 0.1
        if plyr_wlk_idx >= len(player_walk):plyr_wlk_idx = 0
        player_surf = player_walk[int(plyr_wlk_idx)]  

# call animation function before blit the player        
player_animation()         
win.blit(player_surf, player_rect)     

In [ ]:
#05. enemies animation 1
# snail - enemy
snail_frame_1 = pygame.image.load('graphics/snail/snail1.png').convert_alpha()
snail_frame_2 = pygame.image.load('graphics/snail/snail2.png').convert_alpha()
snail_frames = [snail_frame_1, snail_frame_2]
snail_idx = 0
snail_surface = snail_frames[snail_idx]

# fly - enemy
fly_frame_1 = pygame.image.load('graphics/fly/fly1.png').convert_alpha()
fly_frame_2 = pygame.image.load('graphics/fly/fly2.png').convert_alpha()
fly_frames = [fly_frame_1, fly_frame_2]
fly_idx = 0
fly_surf = fly_frames[fly_idx]

In [ ]:
#06. enemy animation 2
# make timers
snail_ani_timer = pygame.USEREVENT + 1
pygame.time.set_timer(snail_ani_timer, 500)

fly_ani_timer = pygame.USEREVENT + 2
pygame.time.set_timer(fly_ani_timer, 200)

# event handling
if event.type == snail_ani_timer:
    if snail_idx == 0: snail_idx = 1
    else: snail_idx = 0
    snail_surface = snail_frames[snail_idx]

if event.type == fly_ani_timer:
    if fly_idx == 0: fly_idx = 1
    else: fly_idx = 0
    fly_surf = fly_frames[fly_idx]

In [ ]:
#07. add sounds

bgm = pygame.mixer.Sound('audio/music.wav')
bgm.play(loops = -1)

jump_sound = pygame.mixer.Sound('audio/jump.mp3')
jump_sound.set_volume(0.5)

# jump action + jump sound
player_gravity = -25
jump_sound.play()

In [ ]:
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.player_walk = []
        self.player_walk.append(pygame.image.load('graphics/player/player_walk_1.png').convert_alpha())
        self.player_walk.append(pygame.image.load('graphics/player/player_walk_2.png').convert_alpha())
        self.plyr_wlk_idx = 0
        self.player_jump = pygame.image.load('graphics/player/jump.png').convert_alpha()        
        
        self.image = self.player_walk[self.plyr_wlk_idx]
        self.rect = self.image.get_rect(midbottom = (200, 300))
        self.gravity = 0
        
    def player_input(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_SPACE] and self.rect.bottom == 300:
            self.gravity = -25
            
    def apply_gravity(self):
        self.gravity += 1
        self.rect.y += self.gravity
        if self.rect.bottom >= 300:
            self.rect.bottom = 300
            
    def animation_state(self):
        if self.rect.bottom < 300:
            self.image = self.player_jump
        else:
            self.plyr_wlk_idx += 0.1
            if self.plyr_wlk_idx >= len(self.player_walk): self.plyr_wlk_idx = 0
            self.image = self.player_walk[int(self.plyr_wlk_idx)]
            
    def update(self):
        self.player_input()
        self.apply_gravity()
        self.animation_state()

In [ ]:
player = pygame.sprite.GroupSingle()
player.add(Player())

player.draw(win)
player.update()

In [ ]:
class Enemy(pygame.sprite.Sprite):
    def __init__(self, type):
        super().__init__()        
        if type == 'fly':
            self.frames = []
            self.frames.append(pygame.image.load('graphics/fly/fly1.png').convert_alpha())
            self.frames.append(pygame.image.load('graphics/fly/fly2.png').convert_alpha())
            y_pos = 210
        else:
            self.frames = []
            self.frames.append(pygame.image.load('graphics/snail/snail1.png').convert_alpha())
            self.frames.append(pygame.image.load('graphics/snail/snail2.png').convert_alpha())
            y_pos = 300
            
        self.ani_idx = 0
        self.image = self.frames[self.ani_idx]
        self.rect = self.image.get_rect(midbottom = (randint(900, 1100), y_pos))       
    
    def animation_state(self):
        self.ani_idx += 0.1
        if self.ani_idx >= len(self.frames): self.ani_idx = 0
        self.image = self.frames[int(self.ani_idx)]   
        
    def update(self):
        self.animation_state()
        self.rect.x -= 6
        self.destroy()
        
    def destroy(self):
        if self.rect.x <= -100:
            self.kill()
        
enemy_group = pygame.sprite.Group()

In [ ]:
from random import randint, choice

if event.type == enemy_timer:
    enemy_group.add(Enemy(choice(['fly', 'snail', 'snail', 'snail'])))